In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
from jupyter_dash import JupyterDash
import pandas as pd

import copy

from pof.failure_mode import FailureMode
from interface.layouts import *

In [2]:
fm = FailureMode().set_default()

In [3]:
# Failure Mode Tests
mfml = make_failure_mode_layout(fm, prefix="fm")

# Task Tests
task = fm.tasks['ocr']
mtl = make_task_layout(task)

mtf = make_task_form(task, prefix="")

# Trigger Tests
triggers= fm.tasks['ocr'].triggers()
mttl = make_task_trigger_layout(triggers)

condition_triggers = fm.tasks['ocr'].triggers()['condition']
mctf = make_condition_trigger_form(condition_triggers)


# Impact Tests

impacts= fm.tasks['ocr'].impacts()
mtil = make_task_impact_layout(impacts)

state_impacts = fm.tasks['ocr'].impacts()['state']
msil = make_state_impact_layout(state_impacts)

condition_impacts = fm.tasks['ocr'].impacts()['condition']
mcil = make_condition_impact_layout(condition_impacts)

condition_impact = fm.tasks['ocr'].impacts()['condition']['wall_thickness']
mtif = make_condition_impact_form(condition_impact)

# Distribution Tests
failure_dist = fm.failure_dist
mdl = make_dist_layout(failure_dist)

In [9]:

# Build App
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
app.layout = html.Div([
    html.Div(children='Tetst Output', id='test_output'),
    dcc.Graph(id="maintenance_strategy"),
    mfml
])

collapses = []
for fm_name, fm in {'fm': fm}.items():
    collapses = collapses + [fm_name]
    for task in fm.tasks:
        collapses = collapses + [fm_name + '-task-' + task]

@app.callback(
    #Output("test_output", "children"),
    [Output(f"{prefix}-collapse", "is_open") for prefix in collapses],
    [Input(f"{prefix}-collapse-button", "n_clicks") for prefix in collapses],
    [State(f"{prefix}-collapse", "is_open") for prefix in collapses],
)
def toggle_collapses(*args):
    ctx = dash.callback_context

    state_id = ""
    collapse_id = ctx.triggered[0]['prop_id'].split('.')[0].replace('-collapse-button','')
    if collapse_id in collapses:

        state_id = collapse_id + '-collapse.is_open'
        ctx.states[state_id] = not ctx.states[state_id]

    is_open = tuple(ctx.states.values())

    return is_open

ms_fig_update = fm.get_dash_ids()
ms_fig_update = ["fm-" + msfu for msfu in ms_fig_update]

@app.callback(
    #Output("maintenance_strategy", "figure"), 
    Output("test_output", "children"),
    [Input(dash_id,"value") for dash_id in ms_fig_update]
)
def update_maintenance_strategy(*args):

    # Check the parameters that changed
    ctx = dash.callback_context
    dash_id = ctx.triggered[0]['prop_id'].split('.')[0]
    value = ctx.triggered[0]['value']

    #Temp
    dash_id = dash_id.replace('fm-', "")

    # Update the model
    #fm_local = copy.deepcopy(fm)
    fm_local =fm
    fm_local.dash_update(dash_id, value)

    fm_local.mc_timeline(t_end=200, n_iterations=100)
    df = fm_local.expected_cost_df()

    fig = px.area(
        df,
        x="time",
        y="cost_cumulative",
        color="task",
        title="Maintenance Strategy Costs",
    )

    #fig.update_yaxes(range=[0, graph_y_limit])

    return fig

#Execute
app.run_server(mode='inline', debug=True)

In [5]:

[Input(dash_id,"value") for dash_id in ms_fig_update]

[<Input `fm-failure_dist-alpha.value`>,
 <Input `fm-failure_dist-beta.value`>,
 <Input `fm-failure_dist-gamma.value`>]

In [6]:
fm.dash_update("failure_dist-alpha", 3)

True

In [7]:
fm.failure_dist.alpha

3

In [8]:
fm.failure_dist.dash_update('alpha', 30)

alpha


True

In [25]:
dash_id

'alpha'